In [0]:
try:
    dbutils.fs.unmount("/mnt/input")
except:
    print("Mount does not exist")

In [0]:
try:
    dbutils.fs.unmount("/mnt/output1")
except:
    print("Mount does not exist")

In [0]:
input_source=dbutils.widgets.get("inputdata")
output_source=dbutils.widgets.get("testoutput")

In [0]:
conf_key="fs.azure.account.key.creditdatastore.blob.core.windows.net"
key_value="z84U/iqdRc38421lLpmrpHZJFgANqnTO8MN5eKzBPfGdav9pm3IPMB9jgou15NH6miTqykdWDvyx+ASt6nxoYQ=="

In [0]:

try:
    dbutils.fs.mount(source=input_source,
                    mount_point="/mnt/input",
                    extra_configs={conf_key:key_value})
except:
    print(" Input Datastore not mounted")
    


In [0]:
try:
    dbutils.fs.mount(source=output_source,
                    mount_point="/mnt/output1",
                    extra_configs={conf_key:key_value})
except:
    
    print("output mounted")
    

In [0]:
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.preprocessing import LabelEncoder
import joblib
LE=LabelEncoder()
credit_dataset_df=pd.read_csv("/dbfs/mnt/input/credit_risk_dataset.csv")
credit_dataset_df['person_emp_length'].fillna(credit_dataset_df['person_emp_length'].median(),inplace=True)
credit_dataset_df['loan_int_rate'].fillna(credit_dataset_df['loan_int_rate'].median(),inplace=True)
x=credit_dataset_df.drop('loan_status',axis=1)
y=credit_dataset_df['loan_status']
x['person_home_ownership']=LE.fit_transform(x['person_home_ownership'])
x['loan_intent']=LE.fit_transform(x['loan_intent'])
x['loan_grade']=LE.fit_transform(x['loan_grade'])
x['cb_person_default_on_file']=LE.fit_transform(x['cb_person_default_on_file'])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=1)
rfc=RandomForestClassifier()
fitted_model=rfc.fit(x_train,y_train)
y_pred=fitted_model.predict(x_test)


In [0]:
dbutils.fs.mkdirs("/mnt/output1/000")

In [0]:
model_file="/dbfs/mnt/output1/rfcmodel.pkl"
joblib.dump(value=fitted_model,filename=model_file)

In [0]:
x_test.to_csv("/dbfs/mnt/output1/x_test.csv", index=False)

y_test = pd.DataFrame(y_test)
y_test.to_csv("/dbfs/mnt/output1/y_test.csv", index=False)

y_pred = pd.DataFrame(y_pred)
y_pred.to_csv("/dbfs/mnt/output1/y_pred.csv",index=False)